<a href="https://colab.research.google.com/github/noahhoang/NEAT-Python-Car/blob/main/neat_python_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install neat-python

import neat # NEAT (NeuroEvolution of Augmenting Topologies) neural network algo
import pygame # rendering game
import os # file pathing
import math # math functions
import random # generating random number
import sys # script handling



import cv2 as cv  # for handling importing images


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.7 MB/s eta 0:00:00
pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Mount Google Drive
Get .png files for **car** and **race track**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# get car image
car = cv.imread('/content/drive/MyDrive/NEAT Python Car/car.png')

# get track image
map = cv.imread('/content/drive/MyDrive/NEAT Python Car/map.png')

## pygame Rendering

In [4]:


# establishing dimensions of pygame
screen_width = 1500
screen_height = 800
generation = 0;



# creating car in pygame
class Car:
  # constructor
    def __init__(self):

      # establishing features of car
      # self.surface = pygame.image.load('car.png') # load car image --- might need to mount google drive for the image
      self.surface = car
      self.surface = pygame.transform.scale(self.surface, (100, 100)) # resize car to 100 pixels
      self.rotate_surface = self.surface
      self.pos = [700, 650]
      self.angle = 0
      self.speed = 0
      self.center = [self.pos[0] + 50, self.pos[1] + 50]
      self.radars = []
      self.radars_for_draw = []
      self.is_alive = True
      self.goal = False
      self.distance = 0
      self.time_spent = 0


      # method to draw the car and its radar in pygame
      def draw(self, screen):
        screen.blit(self.rotate_surface, self.pos)  # put car on screen
        self.draw_radar(screen)

      # draws the radar vectors coming from the car
      def draw_radar(self, screen):
        for radar in self.radars: # draws each radar vector
          pos, dist = radar
          pygame.draw.line(screen, (0, 255, 0), self.center, pos, 1)
          pygame.draw.circle(screen, (0, 255, 0), pos, 5)


      # checks collision status to determine is car is still "alive"
      def check_collision(self, map):
        self.is_alive = True
        for p in self.four_points:  # checks four corners of the car to see
            if map.get_at((int(p[0]), int(p[1]))) == (255, 255, 255, 255):  # if one of the corners is in a white pixel, then it crashed (white track boundaries)
                self.is_alive = False
                break



      # create radar vectors for the cars current position
      def check_radar(self, degree, map):
        len = 0
        # starter x, y coordinate for the radar vector
        x = int(self.center[0] + math.cos(math.radians(360 - (self.angle + degree))) * len)
        y = int(self.center[1] + math.sin(math.radians(360 - (self.angle + degree))) * len)

        # find where the radar vector should end
        while not map.get_at((x, y)) == (255, 255, 255, 255) and len < 300:
          len = len + 1
          x = int(self.center[0] + math.cos(math.radians(360 - (self.angle + degree))) * len)
          y = int(self.center[1] + math.sin(math.radians(360 - (self.angle + degree))) * len)

        dist = int(math.sqrt(math.pow(x - self.center[0], 2) + math.pow(y - self.center[1], 2)))  # calculate distance from car to end of vector
        self.radars.append([x, y], dist)  # store coordinate and the distance from the center of the car



      # updates the car's status as it moves
      def update(self, map):

        # check speed
        self.speed = 15


        # check position - horizontal
        self.rotate_surface = self.rot_center(self.surface, self.angle)
        self.pos[0] += math.cos(math.radians(360 - self.angle)) * self.speed

        # readjust car if too far out of the boundaries
        if self.pos[0] < 20:
          self.post[0] = 20
        elif self.pos[0] > screen_width - 120:
          self.pos[0] = screen_width - 120

        # increment car data
        self.distance += self.speed
        self.time_spent += 1

        # check position - vertical
        self.rotate_surface = self.rot_center(self.surface, self.angle)
        self.pos[1] += math.sin(math.radians(360 - self.angle)) * self.speed

        # readjust car if too far out of the boundaries
        if self.pos[1] < 20:
          self.post[1] = 20
        elif self.pos[1] > screen_width - 120:
          self.pos[1] = screen_width - 120



        # check four corners of contact
        self.center = [self.pos[0] + 50, self.pos[1] + 50]  # re-adjust center
        len = 40  # dist from center to corner of the car
        # calculate the corner points => calculating adjacent and opposite sides of triangle and storing as coordinate points
        left_top = [self.center[0] + math.cos(math.radians(360 - (self.angle + 30))) * len, self.center[1] + math.sin(math.radians(360 - (self.angle + 30))) * len]
        right_top = [self.center[0] + math.cos(math.radians(360 - (self.angle + 150))) * len, self.center[1] + math.sin(math.radians(360 - (self.angle + 150))) * len]
        left_bottom = [self.center[0] + math.cos(math.radians(360 - (self.angle + 210))) * len, self.center[1] + math.sin(math.radians(360 - (self.angle + 210))) * len]
        right_bottom = [self.center[0] + math.cos(math.radians(360 - (self.angle + 330))) * len, self.center[1] + math.sin(math.radians(360 - (self.angle + 330))) * len]
        # store 4 points
        self.four_points = [left_top, right_top, left_bottom, right_bottom]


        # check for a collision
        self.check_collision(map)

        self.radars.clear() # reset radars that were calculated

        # get the new radar vectors for each angle
        for d in range(-90, 120, 45):
          self.check_radar(d, map)


        # updates radar data
        def get_data(self):
          radars = self.radars
          ret = [0, 0, 0, 0, 0]

          # for each radar point
          for i, r in enumerate(radars):
            ret[i] = int(r[1] / 30) # distance of radar / 30 to scale easier for NEAT to process

          return ret


        ## GETTER METHODS ##
        #=================##

        def get_alive(self):
          return self.is_alive

        # establish goal for the car
        def get_reward(self):
          return self.distance / 50.0

        # makes car rotate around its center instead of top-left corner
        def rot_center(self, image, angle):
          orig_rectangle = image.get_rect()
          rot_image = pygame.transform.rotate(image, angle)
          rot_rectangle = orig_rectangle.copy() # gets copy of original car
          rot_rectangle.center = rot_image.get_rect.center  # gets center of rotated car
          rot_image = rot_image.subsurface(rot_rect).copy() # used to crop car to original size
          return rot_image




      # responsible for initializing and running car simulation
      def run_car(genomes, config):

        # initialize params
        nets = []
        cars = []

        for id, g in genomes:
          net = neat.nn.FeedForwardNetwork.create(g, config)
          nets.append(net)
          g.fitness = 0 # establishes how good the neural network is at a task

          # initalize car for this neural network
          cars.append(Car())

        # initialize pygame game
        pygame.init()
        screen = pygame.display.set_mode((screen_width, screen_height))
        clock = pygame.time.Clock()
        generation_font = pygame.font.SysFont("Times New Roman", 70)
        font = pygame.font.SysFont("Times New Roman", 30)

        # import map using mounting google drive
        # map = pygame.image.load('map.png')





        # main loop for the game
        global generation
        generation += 1

        # LOOP
        while True:
          # check to end simulation
          for event in pygame.event.get():
            if event.type == pygame.QUIT:
              sys.exit(0)


          # loop through each car for decision making of turning (left or right)
          for i, car in enumerate(cars):
            output = nets[index].activate(car.get_data()) # passes radar info through neural network to get output vectors
            i = output.index(max(output)) # gets index of most favorable output vector for car

            # logic for turning left vs right
            if i == 0:
              car.angle += 10
            else:
              car.angle -= 10

          # Updates for car and fitness values
          remain_cars = 0
          # cars and genomes are syncrhonized, iterate over the cars to update their respective genome's fitness
          for i, car in enumerate(cars):
            if car.get_alive():
              remain_cars += 1  # count of how many cars are still on the track
              car.update(map) # handles car position and status
              genomes[i][1].fitness += car.get_reward()


          # if there are no cars left
          if remain_cars == 0:
            break

          # draw the simulation
          screen.blit(map, (0,0)) # draw track
          # put each car on the screen
          for car in cars:
            if car.get_alive():
              car.draw(screen)

          # surface to display the generation number
          text = generation_font.render("Generation: " + str(generation), True, (167, 199, 240))
          text_rect = text.get_rect()
          text_rect.center = (screen_width/2, 100)
          screen.blit(text, text_rect)

          # surface to display the remaining car number
          text = render("Remaining cars: " + str(remain_cars), True, (0, 0, 0))
          text_rect = text.get_rect()
          text_rect.center = (screen_width/2, 200)
          screen.blit(text, text_rect)


          pygame.display.flip() # screen frame refreshing
          clock.tick(0) # removes frame rate limit



        # if the script is being run by itself
        if __name__ == "__main__":
          # NEAT config file
          # config_path = "./config-feedforward.txt"
          config_path = cv.imread('/content/drive/MyDrive/NEAT Python Car/neat_car_config_feedforward.txt')

          config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

          # NEAT evolution class
          p = neat.Population(config)

          # statistical reporter
          p.add_reporter(neat.StdOutReporter(True))
          stats = neat.StatisticsReporter()
          p.add_reporter(stats)

          # START PROGRAM - for 1000 generations
          p.run(run_car, 1000)


















